In [ ]:
import pandas as pd

# Cell to parse a FASTA file and produce a two-column table: sequence, identifier

def parse_sequence_txt(path):
    """Yield (identifier, sequence) pairs from a FASTA file."""

    f = open(path, 'r')
    f.readline()
    data = dict()
    data['genus'] = []
    data['species'] = []
    data['identifier'] = []
    data['is_complete'] = []
    data['sequence'] = []
    for line in f:
        line = line.strip().split('\t')
        long = line[0].split(' ')
        data['genus'].append(long[1])
        data['species'].append(long[2])
        data['identifier'].append(long[0])
        data['is_complete'].append(line[1].strip(' '))
        data['sequence'].append(line[2].strip())
    f.close()
    df = pd.DataFrame(data)
    df = df[['genus','species','sequence','identifier','is_complete']]
    return df

# Path to your FASTA file (you wrote "seqence.txt")
input_path = 'lossers/data/sequence.txt'
df = parse_sequence_txt(input_path)
df.head()


             genus      species  \
0      Alitibacter  langaaensis   
1      Alitibacter  langaaensis   
2      Roseovarius    maritimus   
3      Roseovarius       roseus   
4  Planosporangium     spinosum   

                                            sequence   identifier  \
0  ATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCT...  NR_118751.1   
1  ATTGAACGCTGGCGGCAGGCTTAACACATGCAAGTCGAACGGTAAC...  NR_042885.1   
2  CAACTTGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCAGG...  NR_200035.1   
3  CAACTTGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCAGG...  NR_200034.1   
4  TTGTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...  NR_200033.1   

         is_complete  
0   partial sequence  
1   partial sequence  
2  complete sequence  
3  complete sequence  
4  complete sequence  
